In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import RobustScaler

from ombs_senegal.benchmark_model import BenchmarkScores
from ombs_senegal.time_series_deepl import Learner, HydroDataset, split_by_date


DATA_PATH = Path("../../data")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data preprocessing

In [ ]:
df = pd.read_csv(
    DATA_PATH/'data_cumul.csv', 
    sep=';', 
    usecols=['time', 'débit_insitu', 'débit_mgb', 'P_mean'], 
    index_col='time',
    converters={"time": pd.to_datetime}
    )

tamsat_daily_total = xr.load_dataset(DATA_PATH/"tamsat_sub_poly_daily_total.nc")

data = pd.merge(df, tamsat_daily_total["rfe"].to_dataframe(), left_index=True, right_index=True)

data = data["2012-01-01":]
data['month'] = data.index.month
data['month_sin'] = np.sin(2 * np.pi * data.index.month / 12)
data['month_cos'] = np.cos(2 * np.pi * data.index.month / 12)

In [ ]:
def smooth(df, window=7, missing_val=0): return df.rolling(window=window).sum().fillna(missing_val)

data["rfe_7d"] = smooth(data["rfe"])
data["rfe_60d"] = smooth(data["rfe"], 60)
data["imerg_7d"] = smooth(data["P_mean"])

In [ ]:
train, valid, test = split_by_date(data, val_dates=("2018-01-01", "2018-12-31"), test_dates=("2019-01-01", "2020-12-31"))

Now lets define the feature and the target columns and divide data in feature and targets

In [ ]:
x_cols = ["débit_mgb","rfe_7d", "month"]
y_cols = ["débit_insitu"]

x_train, y_train = train[x_cols], train[y_cols]
x_valid, y_valid = valid[x_cols], valid[y_cols]
x_test, y_test = test[x_cols], test[y_cols]

Now we will fit the scaler based only on train data. This ensures that:
1. No information from the validation/test data sets leaks to into the scaling process
2. All data is scaled consistently using the same parameters
3. The model sees new data scaled in the same way as it was trained

In [ ]:
feature_scaler, target_scaler = RobustScaler(), RobustScaler()
_, _ = feature_scaler.fit_transform(x_train), target_scaler.fit_transform(y_train)

## Model definition

Based on the model benchmarking we have already done and on the fact that there is not many available data, we choose a simple Multi Layer Perceptron as model

In [ ]:
class SimpleRegularizedMLP(nn.Module):
    def __init__(self, input_size, prediction_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.norm = nn.LayerNorm(64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, prediction_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.norm(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


## Trainning

In [ ]:

# 🔹 Listes des tailles de fenêtres à tester
context_sizes = [50]
batch_size = 32
learning_rate = 0.0003
epochs=20

prediction_size = 10  # Fixe (peut être ajusté)
x_transform=feature_scaler.transform
y_transform=target_scaler.transform
results = []
models = []

benchmark_scores = BenchmarkScores()

# 🔹 Boucle sur différentes tailles de fenêtres
for context_size in context_sizes:
    print(f"\n🟢 Test avec window_size = {context_size}")

    train_dataset = HydroDataset(x=x_train, y=y_train, ctx_len=context_size, pred_len=prediction_size, x_transform=x_transform, y_transform=y_transform)
    valid_dataset = HydroDataset(x=x_valid, y=y_valid, ctx_len=context_size, pred_len=prediction_size, x_transform=x_transform, y_transform=y_transform)
    test_dataset = HydroDataset(x=x_test, y=y_test, ctx_len=context_size, pred_len=prediction_size, x_transform=x_transform, y_transform=y_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = SimpleRegularizedMLP(len(x_cols)*context_size, prediction_size).to(DEVICE)
    learner = Learner(model=model, train_loader=train_loader, val_loader=valid_loader, verbose=False)
    learner.fit(lr=learning_rate, epochs=epochs)

    y_pred = learner.predict(test_loader, inverse_transform=target_scaler.inverse_transform)

    y_pred.index.name = "time"
    y_pred["model"] = model.__class__.__name__
    y_pred.set_index(["model"], append=True, inplace=True)
    y_pred = y_pred.to_xarray()
    scores = benchmark_scores.compute_scores(y_pred, y_test.to_xarray()[y_cols[0]], metrics=["mae", "rmse", "nse", "kge"])

    mean_scores = {s.upper(): round(float(scores[s].mean().values), 2) for s in scores.data_vars}
    print(f"📊 Résultats pour window_size={context_size} -> {mean_scores}")

    # 🔹 Stocker les résultats
    results.append({"ctx_size": context_size, **mean_scores})

# 🔹 Afficher le meilleur résultat
best = min(results, key=lambda x: x["RMSE"])  # Choix basé sur le RMSE le plus bas
print(f"\n✅ Meilleure fenêtre : {best["ctx_size"]} avec RMSE={best["RMSE"]}, MAE={best["MAE"]}, NSE={best["NSE"]}, KGE={best["KGE"]}")


- MLP raw tamsat: Meilleure fenêtre : 60 avec RMSE=141.604, MAE=73.313, R²=0.887
- MLP smooth 7d tamsat: Meilleure fenêtre : 50 avec RMSE=139.83, MAE=69.64, NSE=0.89, KGE=0.86
- MLP smooth 7d imerg: Meilleure fenêtre : 55 avec RMSE=147.631, MAE=74.793, R²=0.876
- MLP: Meilleure fenêtre : 60 avec RMSE=156.864, MAE=78.964, R²=0.861
- CNN: Meilleure fenêtre : 30 avec RMSE=191.998, MAE=92.670, R²=0.786
- GRU: Meilleure fenêtre : 60 avec RMSE=212.033, MAE=108.959, R²=0.746
- Simple LSTM: Meilleure fenêtre : 90 avec RMSE=197.514, MAE=104.173, R²=0.785

In [ ]:
# 🔹 Fonction pour calculer le PBIAS
def pbias(y_true, y_pred):
    return 100 * np.sum(y_pred - y_true) / np.sum(y_true)

In [ ]:
from ombs_senegal.benchmark_model import plot_prediction_comparison
import matplotlib.pyplot as plt

In [ ]:
_ = plot_prediction_comparison(
    predicted=y_pred.sel(model="SimpleRegularizedMLP"),
    observed=y_test.to_xarray()[y_cols[0]],
    mgb=x_test["débit_mgb"].to_xarray(),
    scores=scores.sel(model="SimpleRegularizedMLP")
)

In [ ]:
scores

In [ ]:
y_pred.sel(model="SimpleRegularizedMLP")

In [ ]:
benchmark_scores = scores.to_array("score", name="scores")
benchmark_results = xr.merge([
    y_pred.to_array("forecast_horizon", name="pred"),
    y_test.to_xarray().rename({"débit_insitu": "obs"})["obs"],
    benchmark_scores
    ])
benchmark_results.to_netcdf(DATA_PATH/'mlp_with_tamsat.nc')